In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [3]:
lam, p, niter = 1e4, 0.01, 10
def baseline_als(y):
    L = len(y)
    D = np.diff(np.eye(L), 2)
    D = lam * D.dot(D.T)
    w = np.ones(L)
    for _ in range(niter):
        b = np.linalg.solve(np.diag(w) + D, w * y)
        w = p * (y > b) + (1 - p) * (y < b)
    return b

def preprocess(arr):
    out = np.zeros_like(arr)
    for i, s in enumerate(arr):
        b = baseline_als(s)
        c = s - b
        norm = np.linalg.norm(c)
        out[i] = c / norm if norm > 0 else c
    return out

In [37]:
def floatify_cols(df):
    new = []
    for c in df.columns:
        if c in ('Label', 'Label 1', 'Label 2'):
            new.append(c)
        else:
            new.append(float(c))
    df.columns = new

In [ ]:
ref_df = pd.read_csv('reference_v2.csv')

def floatify_cols(df):
    new = []
    for c in df.columns:
        if c in ('Label', 'Label 1', 'Label 2'):
            new.append(c)
        else:
            new.append(float(c))
    df.columns = new
floatify_cols(ref_df)

wav_cols   = [c for c in ref_df.columns if c!='Label']
ref_specs  = ref_df[wav_cols].values
ref_labels = ref_df['Label'].values
ref_proc   = preprocess(ref_specs)

In [6]:
from itertools import combinations
# Unique chemical classes
classes    = sorted(np.unique(ref_labels))
C = len(classes)
class_to_i = {c:i for i,c in enumerate(classes)}

# ─── 2) Generate synthetic mixtures ────────────────────────────────────────────
ratios = np.arange(0.05, 1.0, 0.05)
noise_level = 0.01
n_per_ratio = 10  # number of random spectra per pair/ratio

synth_specs = []
synth_labels = []
for (i, ci), (j, cj) in combinations(enumerate(classes), 2):
    # indices of pure spectra for each class
    idx_i = np.where(ref_labels == ci)[0]
    idx_j = np.where(ref_labels == cj)[0]
    for r in ratios:
        for _ in range(n_per_ratio):
            spec_i = ref_specs[np.random.choice(idx_i)]
            spec_j = ref_specs[np.random.choice(idx_j)]
            mix = r * spec_i + (1-r) * spec_j
            mix += np.random.normal(scale=noise_level, size=mix.shape)
            synth_specs.append(mix)
            synth_labels.append((ci, cj))
synth_specs = np.array(synth_specs)        # (n_synth, n_waves)
print("Synthetic raw spectra:", synth_specs.shape)

Synthetic raw spectra: (12540, 1024)


In [7]:
from joblib import Parallel, delayed

def preprocess_single(spectrum):
    """
    Baseline‐correct + L2‐normalize + abs, for one 1D array.
    """
    b = baseline_als(spectrum)
    c = spectrum - b
    norm = np.linalg.norm(c)
    out = c / norm if norm > 0 else c
    return np.abs(out)

# 2) Parallel map over all spectra
#    n_jobs=-1 uses all CPUs; you can set e.g. n_jobs=4 to use 4 cores.
synth_proc = np.vstack(
    Parallel(n_jobs=-1, verbose=10)(
        delayed(preprocess_single)(spec) 
        for spec in synth_specs
    )
)

print("Parallel preprocess done:", synth_proc.shape)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 56 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    7.0s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 153 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 201 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 226 tasks      | elapsed:   11.8s
[Parallel(n_jobs=-1)]: Done 253 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done 309 tasks      | elapsed:  

Parallel preprocess done: (12540, 1024)


[Parallel(n_jobs=-1)]: Done 12540 out of 12540 | elapsed:  4.8min finished


In [16]:
# ─── 4) Split synthetic into train/test for Siamese & evaluation ──────────────
X_train, X_test, y_train_pairs, y_test_pairs = train_test_split(
    synth_proc, synth_labels, test_size=0.2, random_state=42
)

In [23]:
# ─── 5) Dataset for contrastive learning (mixture class as label) ─────────────
class RamanPairDataset(Dataset):
    def __init__(self, specs, pair_labels, augment_fn=None):
        self.specs  = specs
        self.labels = pair_labels
        self.augment = augment_fn
        # index by mixture class
        self.by_label = {}
        for idx, lab in enumerate(pair_labels):
            self.by_label.setdefault(lab, []).append(idx)

    def __len__(self):
        return len(self.specs)

    def __getitem__(self, idx):
        """
        Returns one contrastive pair for the mixture at index `idx`:
        - x1: the mixture spectrum
        - x2: either a positive (same mixture class) or negative (different mixture class) spectrum
        - y: 1.0 if positive pair, 0.0 if negative pair
        """
        import random

        # Anchor: the mixture spectrum
        x1 = self.specs[idx]
        lab1 = self.labels[idx]

        # Randomly decide positive vs. negative pair
        if random.random() < 0.5:
            # Positive pair: pick another instance of the same mixture class
            pos_idx = random.choice(self.by_label[lab1])
            x2 = self.specs[pos_idx]
            y = 1.0
        else:
            # Negative pair: pick a different mixture class
            neg_classes = [l for l in self.by_label.keys() if l != lab1]
            neg_lab = random.choice(neg_classes)
            neg_idx = random.choice(self.by_label[neg_lab])
            x2 = self.specs[neg_idx]
            y = 0.0

        # Apply augmentation if provided
        if self.augment:
            x1 = self.augment(x1)
            x2 = self.augment(x2)

        # Return tensors ready for the Siamese network
        return (
            torch.tensor(x1, dtype=torch.float32).unsqueeze(0),
            torch.tensor(x2, dtype=torch.float32).unsqueeze(0),
            torch.tensor(y, dtype=torch.float32)
        )


In [24]:
def augment(spec, noise_std=0.01, shift_max=2):
    spec_noisy = spec + np.random.normal(0, noise_std, size=spec.shape)
    shift = np.random.randint(-shift_max, shift_max + 1)
    return np.roll(spec_noisy, shift)
train_ds = RamanPairDataset(X_train, y_train_pairs, augment_fn=augment)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

In [25]:
# ─── 6) Siamese network & contrastive loss ────────────────────────────────────
class SiameseNet(nn.Module):
    def __init__(self, input_len, embed_dim=64):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(1,16,7,padding=3), nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(16,32,5,padding=2), nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Flatten(),
            nn.Linear((input_len//4)*32, embed_dim),
            nn.ReLU()
        )
    def forward(self,x):
        z = self.encoder(x)
        return F.normalize(z, dim=1)

def contrastive_loss(z1, z2, label, margin=1.0):
    dist = F.pairwise_distance(z1, z2)
    return (label*dist**2 + (1-label)*F.relu(margin-dist)**2).mean()

In [26]:
# ─── 6) Train the Siamese ──────────────────────────────────────────────────────
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SiameseNet(input_len=ref_proc.shape[1], embed_dim=64).to(device)
opt     = torch.optim.Adam(model.parameters(), lr=1e-3)

In [28]:
# ─── 7) Train Siamese on synthetic mixtures ───────────────────────────────────
for epoch in range(1, 11):
    model.train()
    total = 0.0
    for x1, x2, l in train_loader:
        x1,x2,l = x1.to(device), x2.to(device), l.to(device)
        z1, z2 = model(x1), model(x2)
        loss = contrastive_loss(z1, z2, l)
        opt.zero_grad(); loss.backward(); opt.step()
        total += loss.item()*x1.size(0)
    print(f"Epoch {epoch:03d} Loss: {total/len(train_ds):.4f}")

Epoch 001 Loss: 0.0517
Epoch 002 Loss: 0.0503
Epoch 003 Loss: 0.0499
Epoch 004 Loss: 0.0496
Epoch 005 Loss: 0.0509
Epoch 006 Loss: 0.0525
Epoch 007 Loss: 0.0502
Epoch 008 Loss: 0.0497
Epoch 009 Loss: 0.0502
Epoch 010 Loss: 0.0519


In [29]:
torch.save(model.state_dict(), "siamese_mixture.pth")

In [30]:
# ─── 8) Embed & 1-NN classify synthetic test ─────────────────────────────────
model.eval()
with torch.no_grad():
    train_emb = model(torch.tensor(X_train, dtype=torch.float32).unsqueeze(1).to(device)).cpu().numpy()
    test_emb  = model(torch.tensor(X_test,  dtype=torch.float32).unsqueeze(1).to(device)).cpu().numpy()


In [33]:
# build true multi-hot for synthetic test
C = len(classes)
# Build true multi-hot for synthetic test
Y_true_s = np.zeros((len(X_test), C), dtype=int)
for i, pair in enumerate(y_test_pairs):
    if isinstance(pair, str):
        c1, c2 = pair.split("__")
    else:
        c1, c2 = pair  # already a tuple
    Y_true_s[i, class_to_i[c1]] = 1
    Y_true_s[i, class_to_i[c2]] = 1


# NN classification
# 1-NN classification of synthetic test
Y_pred_s = np.zeros_like(Y_true_s)
for i, z in enumerate(test_emb):
    idx = np.argmin(np.linalg.norm(train_emb - z, axis=1))
    pred_pair = y_train_pairs[idx]
    
    # unpack regardless of tuple or string
    if isinstance(pred_pair, str):
        c1, c2 = pred_pair.split("__")
    else:
        c1, c2 = pred_pair

    Y_pred_s[i, class_to_i[c1]] = 1
    Y_pred_s[i, class_to_i[c2]] = 1


print("\nSynthetic Test Report:")
print(classification_report(Y_true_s, Y_pred_s, target_names=classes, zero_division=0))


Synthetic Test Report:
                              precision    recall  f1-score   support

           1,9-nonanedithiol       0.97      0.97      0.97       425
             1-dodecanethiol       0.91      0.89      0.90       441
             1-undecanethiol       0.88      0.91      0.89       400
        6-mercapto-1-hexanol       0.97      0.97      0.97       431
                     benzene       1.00      1.00      1.00       409
                benzenethiol       0.99      0.99      0.99       396
                        dmmp       1.00      1.00      1.00       439
                        etoh       0.99      1.00      0.99       441
                        meoh       0.99      1.00      0.99       399
       n,n-dimethylformamide       1.00      1.00      1.00       415
                    pyridine       1.00      1.00      1.00       413
tris(2-ethylhexyl) phosphate       0.99      0.98      0.99       407

                   micro avg       0.97      0.97      0.97     

In [40]:
# --- Load mixtures and convert columns ---
mix_df = pd.read_csv('mixtures_dataset.csv')

# Re-use your floatify_cols helper:
def floatify_cols(df):
    new_cols = []
    for c in df.columns:
        if c in ('Label 1', 'Label 2'):
            new_cols.append(c)
        else:
            new_cols.append(float(c))  # convert wavenumber strings → floats
    df.columns = new_cols

floatify_cols(mix_df)

# --- Now select wavenumber columns (they are all numeric) ---
wav_cols = [c for c in mix_df.columns if c not in ('Label 1', 'Label 2')]

# --- Extract spectra as a pure float array ---
mix_specs = mix_df[wav_cols].values.astype(float)  # ensure float64 dtype

# --- Parallel preprocessing now works because mix_specs is numeric ---
mix_proc = np.vstack(
    Parallel(n_jobs=-1, verbose=5)(
        delayed(preprocess_single)(spec) for spec in mix_specs
    )
)
print("Mixtures preprocessed:", mix_proc.shape)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 56 concurrent workers.
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 338 tasks      | elapsed:   14.1s


Mixtures preprocessed: (580, 1024)


[Parallel(n_jobs=-1)]: Done 580 out of 580 | elapsed:   19.1s finished


In [41]:
with torch.no_grad():
    real_emb = model(torch.tensor(mix_proc, dtype=torch.float32).unsqueeze(1).to(device)).cpu().numpy()


In [45]:
# true real multi-hot
Y_true_r = np.zeros((len(mix_df), C), int)
for i,(l1,l2) in enumerate(zip(mix_df['Label 1'], mix_df['Label 2'])):
    Y_true_r[i, class_to_i[l1]] = 1
    Y_true_r[i, class_to_i[l2]] = 1

# classify real
# 1-NN classification of real mixtures
Y_pred_r = np.zeros_like(Y_true_r)
for i, z in enumerate(real_emb):
    idx = np.argmin(np.linalg.norm(train_emb - z, axis=1))
    pred_pair = y_train_pairs[idx]
    if isinstance(pred_pair, str):
        c1, c2 = pred_pair.split("__")
    else:
        c1, c2 = pred_pair

    Y_pred_r[i, class_to_i[c1]] = 1
    Y_pred_r[i, class_to_i[c2]] = 1


# Assuming you’ve already built Y_true_r, Y_pred_r, and classes previously:
support = Y_true_r.sum(axis=0)
valid_idxs = np.where(support > 0)[0]
valid_names = [classes[i] for i in valid_idxs]

Y_true_filtered = Y_true_r[:, valid_idxs]
Y_pred_filtered = Y_pred_r[:, valid_idxs]

print(classification_report(
    Y_true_filtered,
    Y_pred_filtered,
    target_names=valid_names,
    zero_division=0
))


                       precision    recall  f1-score   support

      1-dodecanethiol       0.89      0.54      0.67       243
 6-mercapto-1-hexanol       0.99      0.76      0.86       108
              benzene       1.00      1.00      1.00       193
         benzenethiol       0.50      0.50      0.50        72
                 etoh       0.70      0.70      0.70       121
                 meoh       1.00      0.97      0.98       243
n,n-dimethylformamide       1.00      1.00      1.00        72
             pyridine       1.00      1.00      1.00       108

            micro avg       0.91      0.81      0.86      1160
            macro avg       0.88      0.81      0.84      1160
         weighted avg       0.91      0.81      0.85      1160
          samples avg       0.91      0.81      0.85      1160

